In [1]:
!pip install -U transformers datasets huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 11.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pl

In [2]:
from huggingface_hub import login
from datasets import load_dataset
from transformers import (AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer, DefaultDataCollator, pipeline)
import torch

In [3]:
# authenticate Hugging Face access
HF_TOKEN = os.getenv("HF_TOKEN")

In [4]:
# load SQuAD dataset
squad = load_dataset("squad", split="train[:5000]")
squad = squad.train_test_split(test_size=0.2)# 80/20 split

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.62k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [5]:
# load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForQuestionAnswering.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
# preprocessing function for QA task
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=250,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length"
    )
    offset_mapping = inputs["offset_mapping"]
    answers = examples["answers"]
    start_positions, end_positions = [], []

    for i, offsets in enumerate(offset_mapping):
        answer = answers[i]
        if not answer["answer_start"] or not answer["text"]:
            start_positions.append(0)
            end_positions.append(0)
            continue

        start_char = answer["answer_start"][0]
        end_char = start_char + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # find the start/end of the context in the sequence
        context_start = context_end = None
        for idx, seq_id in enumerate(sequence_ids):
            if seq_id == 1 and context_start is None:
                context_start = idx
            if seq_id == 1:
                context_end = idx

        if (
            context_start is None or context_end is None or
            offsets[context_start][0] is None or offsets[context_end][1] is None or
            offsets[context_start][0] > end_char or offsets[context_end][1] < start_char
        ):
            start_positions.append(0)
            end_positions.append(0)
        else:
            idx = context_start
            while idx <= context_end and offsets[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offsets[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    inputs["example_id"] = examples["id"]
    return inputs

In [7]:
# apply preprocessing to dataset
tokenized_squad = squad.map(preprocess_function, batched=True, remove_columns=squad["train"].column_names)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [8]:
#training arguments
training_args = TrainingArguments(
    output_dir="bert_base_squad_model",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=False,
    report_to="none"
)

In [9]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 9.9 MB/s eta 0:00:00


In [10]:
# define evaluation postprocessing
from collections import OrderedDict
import numpy as np
import evaluate

metric = evaluate.load("squad")

In [11]:
# helps pad the batches automatically
data_collator = DefaultDataCollator()

In [12]:
def postprocess_qa_predictions(examples, features, raw_predictions, tokenizer):
    all_start_logits, all_end_logits = raw_predictions
    example_id_to_index = {k["id"]: i for i, k in enumerate(examples)}
    features_per_example = {}
    for i, feature in enumerate(features):
        example_id = feature["example_id"]
        features_per_example.setdefault(example_id, []).append(i)

    predictions = OrderedDict()
    for example_id, feature_indices in features_per_example.items():
        context = examples[example_id_to_index[example_id]]["context"]
        best_score, answer = -float("inf"), ""
        for i in feature_indices:
            start_logits, end_logits = all_start_logits[i], all_end_logits[i]
            offset_mapping = features[i]["offset_mapping"]
            start_idx, end_idx = np.argmax(start_logits), np.argmax(end_logits)
            if start_idx >= len(offset_mapping) or end_idx >= len(offset_mapping):
                continue
            start_char, end_char = offset_mapping[start_idx][0], offset_mapping[end_idx][1]
            if start_char is None or end_char is None:
                continue
            current_answer = context[start_char:end_char]
            score = start_logits[start_idx] + end_logits[end_idx]
            if score > best_score:
                best_score = score
                answer = current_answer
        predictions[example_id] = answer
    return predictions

In [13]:
def compute_metrics(eval_preds):
    predictions, _ = eval_preds
    final_predictions = postprocess_qa_predictions(
        examples=squad["test"],
        features=tokenized_squad["test"],
        raw_predictions=predictions,
        tokenizer=tokenizer
    )
    formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
    references = [{"id": ex["id"], "answers": ex["answers"]} for ex in squad["test"]]
    return metric.compute(predictions=formatted_predictions, references=references)

In [14]:
# init Trainer with evaluation metrics
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_squad["train"],
    eval_dataset=tokenized_squad["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

<ipython-input-14-f1bc759b87e7>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [15]:
# train and evaluate
trainer.train()
results = trainer.evaluate()
print(results)

Step,Training Loss
500,2.643300
1000,1.241000
1500,0.798900


{'eval_loss': 1.3319123983383179, 'eval_exact_match': 57.4, 'eval_f1': 67.64041338878008, 'eval_runtime': 17.8074, 'eval_samples_per_second': 56.156, 'eval_steps_per_second': 7.02, 'epoch': 3.0}


In [16]:
trainer.save_model("bert_base_squad_model_final")

question_answerer = pipeline(
    "question-answering",
    model="bert_base_squad_model_final",
    tokenizer=tokenizer
)

Device set to use cuda:0


In [17]:
# to test model predictions
def ask_question(question, context):
    print(f"\nQ: {question}")
    print(f"Context: {context}")
    answer = question_answerer(question=question, context=context)
    print(f"Answer: {answer['answer']}")

In [18]:
ask_question(
    "How many programming languages does BLOOM support?",
    "BLOOM has 176 billion parameters and can generate text in 46 natural languages and 13 programming languages."
)


Q: How many programming languages does BLOOM support?
Context: BLOOM has 176 billion parameters and can generate text in 46 natural languages and 13 programming languages.
Answer: 13


In [19]:
ask_question("When did Chomsky earn his PhD?",
             "During his postgraduate work in the Harvard Society of Fellows, Chomsky developed the theory of transformational grammar for which he earned his doctorate in 1955.")


Q: When did Chomsky earn his PhD?
Context: During his postgraduate work in the Harvard Society of Fellows, Chomsky developed the theory of transformational grammar for which he earned his doctorate in 1955.
Answer: 1955


In [20]:
ask_question("Where is the University of Hull?",
             "The main university campus is located in Hull and is home to the Hull York Medical School, a joint initiative with the University of York.")


Q: Where is the University of Hull?
Context: The main university campus is located in Hull and is home to the Hull York Medical School, a joint initiative with the University of York.
Answer: Hull York Medical School


In [21]:
ask_question("When did harlequin rasboras first become popular?",
             "The species became an instant favorite among aquarists after its introduction in the early 1900s and is the best known and most widely kept species among the rasboras.")


Q: When did harlequin rasboras first become popular?
Context: The species became an instant favorite among aquarists after its introduction in the early 1900s and is the best known and most widely kept species among the rasboras.
Answer: 1900s


In [22]:
ask_question("What does UNFCCC stand for?",
             "The 2023 United Nations Climate Change Conference or Conference of the Parties of the UNFCCC, more commonly referred to as COP28, is the 28th United Nations Climate Change conference, being held from 30 November until 12 December 2023 at Expo City, Dubai.")


Q: What does UNFCCC stand for?
Context: The 2023 United Nations Climate Change Conference or Conference of the Parties of the UNFCCC, more commonly referred to as COP28, is the 28th United Nations Climate Change conference, being held from 30 November until 12 December 2023 at Expo City, Dubai.
Answer: Conference of the Parties


In [23]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [24]:
trainer.save_model("/content/drive/MyDrive/bert_base_squad_model")